<a href="https://colab.research.google.com/github/wyrzykow/BHTOM-utils/blob/main/BHTOM_for_Staszek.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#BHTOM newletter for CDK-500 Krakow telescope

In [174]:
!pip install astroplan

In [175]:
from astropy.coordinates import get_body, SkyCoord
from astropy import units
from astropy.time import Time
from astroplan import Observer, FixedTarget, time_grid_from_range
import numpy as np
from datetime import datetime, timedelta, date


In [176]:
#SETUP
send_email=True

#telescope setup:
site_details = {'sitecode': 'Krakow', 'latitude': 50.05, 'longitude': 19.82, 'elevation': 300}
camera_name = "OAUJ-CDK500_U47" ##TODO: what if two cameras are used? #OAUJ-CDK500_U47 , OAUJ-CDK500_F42
mag_limit = 17.0
dec_limit = -30
destination_email = "staszek.zola@gmail.com"

In [ ]:
#visibility parameters:
start_time = datetime.now()

airmass_limit = 2.5
length=2 #in days
interval=60 #in minutes between samplings
observer = Observer(longitude=site_details.get('longitude')*units.deg,
                    latitude=site_details.get('latitude')*units.deg,
                    elevation=site_details.get('elevation')*units.m)

In [177]:
#BHTOM token
from google.colab import userdata
auth_token=userdata.get('bhtomtoken')

In [178]:
def get_astroplan_sun_and_time(start_time, end_time, interval):
    """
    Uses astroplan's time_grid_from_range to generate
    an astropy Time object covering the time range.

    Uses astropy's get_sun to generate sun positions over
    that time range.

    If time range is small and interval is coarse, approximates
    the sun at a fixed position from the middle of the
    time range to speed up calculations.
    Since the sun moves ~4 minutes a day, this approximation
    happens when the number of days covered by the time range
    * 4 is less than the interval (in minutes) / 2.

    :param start_time: start of the window for which to calculate the airmass
    :type start_time: datetime

    :param end_time: end of the window for which to calculate the airmass
    :type end_time: datetime

    :param interval: time interval, in minutes, at which to calculate airmass within the given window
    :type interval: int

    :returns: ra/dec positions of the sun over the time range,
        time range between start_time and end_time at interval
    :rtype: astropy SkyCoord, astropy Time
    """

    start = Time(start_time)
    end = Time(end_time)

    time_range = time_grid_from_range(time_range=[start, end], time_resolution=interval*units.minute)

    number_of_days = end.mjd - start.mjd
    if number_of_days*4 < float(interval)/2:
        # Hack to speed up calculation by factor of ~3
        sun_coords = get_body("sun",time_range[int(len(time_range)/2)])
        sun = FixedTarget(name='sun', coord=SkyCoord(sun_coords.ra, sun_coords.dec, unit='deg'))
    else:
        sun = get_body("sun", time_range)

    return sun, time_range


In [179]:

#for a given target and observatory, returns true if the object is visible in the period provided
def check_visibility(name, ra, dec, observer, start_time, length=2, interval=60, airmass_limit=2.5):
  body = FixedTarget(name=name, coord=SkyCoord(ra, dec, unit='deg'))

  end_time = start_time + timedelta(days=length)

  visibility = {}
  sun, time_range = get_astroplan_sun_and_time(start_time, end_time, interval)
  sun_alt = observer.altaz(time_range, sun).alt
  obj_airmass = observer.altaz(time_range, body).secz

  bad_indices = np.argwhere(
      (obj_airmass >= airmass_limit) |
      (obj_airmass <= 1) |
      (sun_alt > -12*units.deg)  # between astronomical twilights, i.e. sun is up
  )

  obj_airmass = [None if i in bad_indices else float(airmass) for i, airmass in enumerate(obj_airmass)]

  # Check if all elements in obj_airmass are None
  if all(i is None for i in obj_airmass):
#      print("The list obj_airmass is full of None values.")
      return False
  else:
#      print("The list obj_airmass contains non-None values.")
      return True


In [180]:
# name="Gaia24bfe"
# ra,dec=311.33472,-17.57744

# check_visibility(name, ra, dec, observer, start_time)

In [181]:
import requests
import pandas as pd
import json
from datetime import datetime, timedelta
import pytz

from IPython.display import display, HTML


In [182]:
 #asking for ALL targets, as for now, API does not allow for filtering on importance
request_body = {
         "decMin": dec_limit

    }

# Define headers
headers = {
    'accept': 'application/json',
    'Authorization': f'Token {auth_token}',  ## you can hard-code your token here as well
    'Content-Type': 'application/json',
    'X-CSRFToken': 'uUz2fRnXhPuvD9YuuiDW9cD1LsajeaQnE4hwtEAfR00SgV9bD5HCe5i8n4m4KcOr'
}
api_url =  "https://bh-tom2.astrolabs.pl/targets/getTargetList/"

# Send the POST request
response = requests.post(api_url, json=request_body, headers=headers)
data_json = json.loads(response.text)
data = json.loads(data_json)
data_list = [item['fields'] for item in data]
df = pd.DataFrame(data_list)

#print(df)


In [183]:
df.columns

Index(['name', 'type', 'created', 'modified', 'ra', 'dec', 'epoch', 'parallax',
       'pm_ra', 'pm_dec', 'galactic_lng', 'galactic_lat', 'distance',
       'distance_err', 'scheme', 'epoch_of_elements', 'mean_anomaly',
       'arg_of_perihelion', 'eccentricity', 'lng_asc_node', 'inclination',
       'mean_daily_motion', 'semimajor_axis', 'epoch_of_perihelion',
       'ephemeris_period', 'ephemeris_period_err', 'ephemeris_epoch',
       'ephemeris_epoch_err', 'perihdist', 'classification', 'discovery_date',
       'mjd_last', 'mag_last', 'importance', 'cadence', 'priority',
       'sun_separation', 'creation_date', 'constellation', 'dont_update_me',
       'phot_class', 'photometry_plot', 'photometry_plot_obs',
       'photometry_icon_plot', 'spectroscopy_plot', 'data_plot', 'filter_last',
       'cadence_priority', 'description'],
      dtype='object')

In [184]:
#filtering on importance and sundistance

df1 = df[(df.importance >= 1) & (df.sun_separation > 50) & (df.mag_last<mag_limit)].copy().reset_index(drop=True)

print(len(df1))



40


In [185]:
current_date = datetime.now(pytz.UTC)
formatted_date = current_date.strftime("%d %B, %Y")



In [186]:
## Selection of targets:
# limit the list to visible now
visibility_results = []

# Iterate over DataFrame rows
for index, row in df1.iterrows():
    # Apply check_visibility function and append result to list
    visibility_result = check_visibility(row['name'], row['ra'], row['dec'], observer, start_time)
    visibility_results.append(visibility_result)

# Add results as a new column to the DataFrame
df1['visibility'] = visibility_results
df_visible = df1[df1['visibility'] == True]
df_visible = df_visible.reset_index(drop=True)

df_visible

,name,type,created,modified,ra,dec,epoch,parallax,pm_ra,pm_dec,...,phot_class,photometry_plot,photometry_plot_obs,photometry_icon_plot,spectroscopy_plot,data_plot,filter_last,cadence_priority,description,visibility
0,Gaia23crr,SIDEREAL,2023-10-09T08:35:24.142Z,2023-10-23T15:25:19.085Z,298.123220,29.443690,2000.0,None,None,None,...,Be Star 84.8%,/plots/photometry/Gaia23crr_plot.json,/plots/photometry/obs_Gaia23crr_plot.json,/plots/photometryIcon/Gaia23crr_plot.json,,None,ATLAS(o),0.0,candidate Be-star outburst,True
1,ZTF18aarippg,SIDEREAL,2023-10-09T10:27:02.697Z,2023-10-09T10:27:02.697Z,217.566838,23.062372,2000.0,None,None,None,...,None,/plots/photometry/ZTF18aarippg_plot.json,/plots/photometry/obs_ZTF18aarippg_plot.json,/plots/photometryIcon/ZTF18aarippg_plot.json,,None,ATLAS(o),19.8,Tick-Tok possibly merging Super Massive Black ...,True
2,Gaia23cpd,SIDEREAL,2023-10-12T11:33:02.987Z,2024-06-03T15:34:38.122Z,287.536760,-4.720760,2000.0,None,None,None,...,Red Giant 99.6%,/plots/photometry/Gaia23cpd_plot.json,/plots/photometry/obs_Gaia23cpd_plot.json,/plots/photometryIcon/Gaia23cpd_plot.json,,None,ATLAS(o),10.8,potential long and bright microlensing event o...,True
3,Gaia18bad,SIDEREAL,2023-10-24T14:40:42.556Z,2024-02-07T12:11:11.482Z,331.764940,5.890000,2000.0,None,None,None,...,Ulens Candidate 70.8%,/plots/photometry/Gaia18bad_plot.json,/plots/photometry/obs_Gaia18bad_plot.json,/plots/photometryIcon/Gaia18bad_plot.json,,None,~G,5.8,unknown,True
4,Gaia21asp,SIDEREAL,2023-10-24T14:46:23.888Z,2024-01-08T10:36:37.735Z,292.135720,19.836640,2000.0,None,None,None,...,Be Star 56.4%,/plots/photometry/Gaia21asp_plot.json,/plots/photometry/obs_Gaia21asp_plot.json,/plots/photometryIcon/Gaia21asp_plot.json,,None,ATLAS(o),0.1,"unknown slowly rising object, probably Be-type...",True
5,Gaia23bsf,SIDEREAL,2023-10-25T17:24:20.288Z,2024-06-03T15:35:46.299Z,276.583080,-14.036970,2000.0,None,None,None,...,YSO 51.4%,/plots/photometry/Gaia23bsf_plot.json,/plots/photometry/obs_Gaia23bsf_plot.json,/plots/photometryIcon/Gaia23bsf_plot.json,,None,ATLAS(o),9.7,slowly rising object,True
6,B1721+343,SIDEREAL,2023-11-13T15:06:28.337Z,2023-11-20T15:54:41.813Z,260.836704,34.299578,2000.0,None,None,None,...,Be Star 89.8%,/plots/photometry/B1721+343_plot.json,/plots/photometry/obs_B1721+343_plot.json,/plots/photometryIcon/B1721+343_plot.json,,None,ATLAS(o),0.1,A giant QSO,True
7,RZ_LMi,SIDEREAL,2023-11-14T13:40:56.720Z,2024-01-22T17:47:13.609Z,147.953860,34.123324,2000.0,None,None,None,...,Be Star 100.0%,/plots/photometry/RZ_LMi_plot.json,/plots/photometry/obs_RZ_LMi_plot.json,/plots/photometryIcon/RZ_LMi_plot.json,,None,ATLAS(o),21.4,None,True
8,V374_Peg,SIDEREAL,2023-11-19T14:56:43.389Z,2023-11-20T10:08:32.364Z,330.304684,28.306919,2000.0,None,None,None,...,Evolved 53.2%,/plots/photometry/V374_Peg_plot.json,/plots/photometry/obs_V374_Peg_plot.json,/plots/photometryIcon/V374_Peg_plot.json,,None,ATLAS(o),4.0,None,True
9,8C0716_714,SIDEREAL,2023-11-20T23:13:38.056Z,2024-01-08T15:20:08.123Z,110.472701,71.343434,NaN,None,None,None,...,Be Star 81.8%,/plots/photometry/8C0716_714_plot.json,/plots/photometry/obs_8C0716_714_plot.json,/plots/photometryIcon/8C0716_714_plot.json,,None,R(GaiaSP),374.2,high cadence variability suspected,True


In [187]:
#asking for ALL dataproducts from this telescope

from datetime import datetime
from dateutil.relativedelta import relativedelta

# Get the current date and time
now = datetime.now()

start_date = "2023-01-01T00:00:00.00+00:00"

request_body = {
#    "created_start": week_ago_str
    "created_start": start_date,
    "camera": camera_name
}

# Define headers
headers = {
    'accept': 'application/json',
    'Authorization': f'Token {auth_token}',  ## you can hard-code your token here as well
    'Content-Type': 'application/json',
    'X-CSRFToken': 'uUz2fRnXhPuvD9YuuiDW9cD1LsajeaQnE4hwtEAfR00SgV9bD5HCe5i8n4m4KcOr'
}
api_url =  "https://bh-tom2.astrolabs.pl/common/api/data/"

response = requests.post(api_url, json=request_body, headers=headers)
data_json = json.loads(response.text)
df_data = pd.DataFrame(data_json)


In [188]:
#removing Dry Runs
df_data.drop(df_data[df_data['dryRun'] == True].index, inplace=True)

df_data.columns

Index(['id', 'user_name', 'target_name', 'target', 'user', 'camera',
       'observatory_name', 'observatory', 'product_id', 'data', 'status',
       'photometry_data', 'fits_data', 'extra_data', 'created', 'modified',
       'data_product_type', 'featured', 'thumbnail', 'dryRun', 'comment',
       'observation_record', 'group'],
      dtype='object')

In [189]:
total_fits = len(df_data)
total_fits

449

In [190]:
#Last week's yield:
df_data['created'] = pd.to_datetime(df_data['created'])
one_week_ago = datetime.now(pytz.UTC) - timedelta(weeks=1)
df_data_last_week = df_data[df_data['created'] > one_week_ago]
last_week_counts = len(df_data_last_week)
last_week_counts

6

In [191]:
#Last week's success rate:
# Get counts of each status
status_counts = df_data_last_week['status'].value_counts()

# Print counts
print(status_counts)

# Count for success statuses
success_count = status_counts.get('S', 0)
print(f'Success count: {success_count}')

# Count for error statuses
error_count = status_counts.get('E', 0)
print(f'Error count: {error_count}')

# Count for other statuses
other_count = status_counts.sum() - success_count - error_count
print(f'Other statuses count: {other_count}')

status
P    4
E    2
Name: count, dtype: int64
Success count: 0
Error count: 2
Other statuses count: 4


In [192]:
#Extracting a list of targets observed with this telescope ever

unique_targets = df_data['target_name'].value_counts()

unique_targets

target_name
SN2023ixf    291
Gaia23crr     76
TCrB          47
Gaia23dai     29
Gaia24ayd      6
Name: count, dtype: int64

In [193]:
#Scoring system:
# name contains Gaia = +5
# phot_class contains Ulens or Red Giant = +5
# description contains microlensing = +5
# classification contains Microlensing = +5
# description contains COLIBRI = +2
# description contains nova = +2
#
# AND +10 if the target was already observed with this telescope

def score(row):
    points = row['importance']
    if row['name'] and 'Gaia' in row['name']:
        points += 5
    if row['phot_class'] and ('Ulens' in row['phot_class'] or 'Red Giant' in row['phot_class']):
        points += 3
    if row['description'] and 'microlensing' in row['description']:
        points += 5
    if row['classification'] and 'Microlensing' in row['classification']:
        points += 5
    if row['description'] and 'COLIBRI' in row['description']:
        points += 2
    if row['description'] and 'nova' in row['description']:
        points += 2
    if row['name'] in unique_targets:
        points +=100
    return points

df_visible['score'] = df_visible.apply(score, axis=1)

df_sorted = df_visible.sort_values('score', ascending=False)
#df_sorted

In [194]:
from astropy.coordinates import SkyCoord
import astropy.units as u

def convert_ra_to_hex(row):
    c = SkyCoord(ra=row['ra']*u.degree, dec=row['dec']*u.degree, frame='icrs')
    ra_hms = c.ra.to_string(u.hourangle, sep=":", precision=2)
    return ra_hms

df_sorted['ra_hex'] = df_sorted.apply(convert_ra_to_hex, axis=1)

def convert_dec_to_sex(row):
    c = SkyCoord(ra=row['ra']*u.degree, dec=row['dec']*u.degree, frame='icrs')
    dec_dms = c.dec.to_string(u.degree, sep=":", alwayssign=True, precision=2)
    return dec_dms

df_sorted['dec_hex'] = df_sorted.apply(convert_dec_to_sex, axis=1)

#df_sorted

In [195]:
#OUTPUTING all target found:
from astropy.coordinates import SkyCoord
import astropy.units as u

# Selecting the required columns
df_selected = df_sorted[['name', 'ra_hex', 'dec_hex', 'mag_last', 'sun_separation', 'classification', 'description', 'score']]

# Define a function to format the strings
def make_hyperlink(name):
    return '<a href="https://bh-tom2.astrolabs.pl/targets/{0}/">{0}</a>'.format(name)

df_selected = df_selected.copy()
df_selected.loc[:, 'name'] = df_selected['name'].apply(make_hyperlink)

# Convert the DataFrame to HTML and escape=False to render HTML tags
html_table = df_selected.to_html(index=False, escape=False)
# Remove border by replacing <table> with <table border="0">
html_table = html_table.replace('<table', '<table border="0"')



#formating the email

In [196]:
# Creating the email message
email_message = f"""
Hello {camera_name},
<p>
Greetings from the BHTOM Automated Target Selector!
</p>
<p>

</p>
<p>
Date: {formatted_date}<br>
Targets for you (importance>=1, mag_limit {mag_limit}, visible now, you observed it already are at the top)</p>
<p>
{html_table}
</p>
<hr>
<p>
<b>Last week's count {last_week_counts} ({success_count} with success).</b>
<br>Your observatory collected in total: {total_fits} - thank you!
<hr>
Clear skies!
<br>
Your BHTOM Team
<br>
<a href="https://bhtom.space">bhtom.space</a>
</p>
"""

In [197]:
(email_message)

'\nHello OAUJ-CDK500_U47,\n<p>\nGreetings from the BHTOM Automated Target Selector!\n</p>\n<p>\n\n</p>\n<p>\nDate: 05 June, 2024<br>\nTargets for you (importance>=1, mag_limit 17.0, visible now, you observed it already are at the top)</p>\n<p>\n<table border="0" border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th>name</th>\n      <th>ra_hex</th>\n      <th>dec_hex</th>\n      <th>mag_last</th>\n      <th>sun_separation</th>\n      <th>classification</th>\n      <th>description</th>\n      <th>score</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <td><a href="https://bh-tom2.astrolabs.pl/targets/Gaia24ayd/">Gaia24ayd</a></td>\n      <td>20:03:18.02</td>\n      <td>+30:39:04.54</td>\n      <td>13.2</td>\n      <td>110.0</td>\n      <td>Unknown</td>\n      <td>bright candidate for microlensing event</td>\n      <td>120.0</td>\n    </tr>\n    <tr>\n      <td><a href="https://bh-tom2.astrolabs.pl/targets/TCrB/">TCrB</a></td>\n      <td>15:59:30.1

In [198]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from getpass import getpass

if send_email:
  # Set up the SMTP server
  # Set up the SMTP server
  s = smtplib.SMTP(host='smtp.gmail.com', port=587)
  s.starttls()

  # Login to your Gmail account
  password = 'ggzr clob tbcm ehtc'
  s.login('wyrzykow@gmail.com', password)

  # Create a message
  msg = MIMEMultipart()  # create a message

  # Setup the parameters of the message
  msg['From']="wyrzykow@gmail.com"
  msg['To']=destination_email
  msg['Subject']=f"BHTOM Targets for {formatted_date} for {camera_name}"

  msg.attach(MIMEText(email_message, 'html'))

  # Send the message via the server set up earlier.
  s.send_message(msg)

  # Terminate the SMTP session and close the connection
  s.quit()


In [199]:
### Separate code for BHTOM webpage:
# recently observed events - target names
# one-year statistics: winners per users


In [200]:
display(HTML(email_message))

name,ra_hex,dec_hex,mag_last,sun_separation,classification,description,score
Gaia24ayd,20:03:18.02,+30:39:04.54,13.2,110.0,Unknown,bright candidate for microlensing event,120.0
TCrB,15:59:30.16,+25:55:12.61,10.2,130.0,Nova,recurent nova predicted to explode 2024/2025,115.0
Gaia23crr,19:52:29.57,+29:26:37.28,12.6,112.0,Be-star outburst,candidate Be-star outburst,106.0
Gaia23cpd,19:10:08.82,-4:43:14.74,15.1,142.0,Microlensing Event,potential long and bright microlensing event or weird variable,24.0
Gaia24bfe,20:45:20.33,+27:34:38.78,16.7,105.0,Unknown,candidate microlensing event,20.0
Gaia23dfy,18:47:41.43,+9:02:38.29,16.5,138.0,Unknown,candidate microlensing event,20.0
Gaia24azc,19:44:48.53,+23:37:50.88,14.8,118.0,Unknown,bright gal.plane source candidate microlensing event or Be-type outburst,20.0
Gaia24alm,22:45:19.16,+60:55:08.76,16.0,71.0,Unknown,candidate bright microlensing event on the rise or Be star,18.0
ZTF23aaqazhf,19:01:32.48,+22:36:53.91,15.3,125.0,Unknown,"potential microlensing event from FINK, KATS24K002",16.0
AT2024hcr,18:17:34.80,-7:42:55.40,16.5,155.0,Microlensing Event,"candidate microlensing events from Chinese KATS survey, AT 2024hcr = ZTF24aahkoms = KATS24H004, emailed by Wenjie Zhou",15.0
